## Correction de l'exercice : test buggué + test de Student

### Identification du bug

Le bug de la fonction :

On procède à un **test unilatéral au lieu de bilatéral** : on ne teste que la queue gauche (`p = n.cdf(mean_tested)`) !!!

Par ailleurs, comme rappelé ans le cours, attention la loi normale est une mauvaise distribution pour tester une moyenne : pour la démonstration de la logique du test d’hypothèse on a utilisé `norm` (loi normale) au lieu de `t` (loi de Student)

### Code pour un test de Student bilatéral

```python
from scipy.stats import t  # On utilise la loi de Student
import numpy as np
import matplotlib.pyplot as plt

alpha = 0.05
mean_tested = 2900
mu_0 = 3000  # moyenne sous H0
sigma = 500
n_sample = 100

# Calcul de la statistique de test
t_stat = (mean_tested - mu_0) / (sigma / np.sqrt(n_sample))

# Distribution de Student avec n-1 degrés de liberté
ddl = n_sample - 1
distribution = t(df=ddl, loc=0, scale=1)

# Test bilatéral : on calcule la p-value des deux côtés
p_value = 2 * distribution.cdf(t_stat)  # multiplication par 2 pour test bilatéral

# Condition de rejet correcte
if p_value <= alpha:
    print(f"L'hypothèse H0 est rejetée avec p={p_value:.4f} et α={alpha}")
else:
    print(f"L'hypothèse H0 ne peut pas être rejetée avec p={p_value:.4f} et α={alpha}")

# Visualisation de la distribution de la statistique de test
x = np.linspace(-4, 4, 1000)
pdf = distribution.pdf(x)

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(x, pdf, label='Distribution de Student (ddl=99)')
ax.axvline(t_stat, color='red', linestyle='--', label=f't observé = {t_stat:.2f}')
ax.set_title("Distribution de la statistique de test sous H0")
ax.set_xlabel("Valeur de t")
ax.set_ylabel("Densité")
ax.legend()
plt.show()

# Visualisation de la p-value (zones de rejet)
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(x, pdf, color='blue', label='Distribution t(99)')

# Zones de rejet bilatérales
t_critique = distribution.ppf(alpha/2)  # quantile à gauche
mask_left = x <= t_critique
mask_right = x >= -t_critique
ax.fill_between(x[mask_left], 0, pdf[mask_left], color='red', alpha=0.3, label=f'Zone de rejet (α/2)')
ax.fill_between(x[mask_right], 0, pdf[mask_right], color='red', alpha=0.3)

# Position de la statistique observée
ax.axvline(t_stat, color='darkred', linestyle='--', linewidth=2, label=f't observé = {t_stat:.2f}')

ax.set_title(f"Test bilatéral : p-value = {p_value:.4f}")
ax.set_xlabel("Valeur de t")
ax.set_ylabel("Densité")
ax.legend()
plt.show()
```

### Rappels

**Pourquoi la loi de Student ?**
- Quand on estime l'écart-type à partir d'un échantillon, la distribution de la statistique $t = \frac{\bar{x} - \mu_0}{s/\sqrt{n}}$ suit une loi de Student à $n-1$ degrés de liberté (et non une loi normale)

**Pourquoi un test bilatéral ?**
- On teste $H_0: \mu = 3000$ contre $H_1: \mu \neq 3000$
- Il faut donc considérer les deux queues de la distribution
- La p-value est : $p = 2 \times P(T \leq t_{obs})$ si $t_{obs} < 0$